In [1]:
import os 

In [2]:
%pwd

'd:\\per\\Codes\\Kaggle\\Maize-Disease-Detection-with-Computer-Vision\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\per\\Codes\\Kaggle\\Maize-Disease-Detection-with-Computer-Vision'

In [5]:
# credentials in the environment variable
os.environ["MLFLOW_TRACKING_URI"] = 'https://dagshub.com/Joshua-Abok/Maize-Disease-Detection-with-Computer-Vision.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"] = 'Joshua-Abok'
os.environ["MLFLOW_TRACKING_PASSWORD"] = [password]

In [6]:
import tensorflow as tf

In [7]:
# load model
model = tf.keras.models.load_model("artifacts/training/model.h5")

##### 4. update/initialize entities

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

##### 5. update configuration manager in src config

In [9]:
from src.maizeDiseaseDetection.constants import * 
from src.maizeDiseaseDetection.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager: 
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH): 

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5", 
                training_data="artifacts/data_ingestion/Pictures", 
                mlflow_uri="https://dagshub.com/Joshua-Abok/Maize-Disease-Detection-with-Computer-Vision.mlflow",
                all_params=self.params, 
                params_image_size=self.params.IMAGE_SIZE, 
                params_batch_size=self.params.BATCH_SIZE
          )
          return eval_config

##### 6. Update components

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation: 
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1], 
            batch_size=self.config.params_batch_size, 
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self._valid_generator =  valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation", 
            shuffle=False, 
            **dataflow_kwargs
        )



    # staticmethod decorator to load the model
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    #after loading the model, now evaluate the model
    def evaluation(self): 
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()   #generate validation dataset
        self.score = model.evaluate(self._valid_generator)   # evaluate model, as taking the valid_generator
                                                            # and determine some score. set the score
        self.save_score()   # calling the save_score method

    # save the score 
    def save_score(self): 
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)   # saving scores in json file


    # mlflow integration
    def log_into_mlflow(self): 
        mlflow.set_registry_uri(self.config.mlflow_uri) # uri track the server, will take the lin
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():    # start tracking the experiment
            mlflow.log_params(self.config.all_params)  # log the params from config file
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}  # log metrics loss, accuracy
            )

            # Model registry does not work with file store 
            if tracking_uri_type_store != "file":
                # Register the model 
                # There are other ways to use the Model Registry, which depends on the use case, 
                # please refer to the doc for more information: 
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else: 
                mlflow.keras.log_model(self.model, "model")


                                            


##### 7. initialize/update the pipeline

In [15]:
#defining pipeline would mean: 
    #  1. calling the evaluation() function
    #  2. calling the mlflow tracking -> log_into_mlflow()

try: 
    config = ConfigurationManager()   #passing training config in the training
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()   # logging experiment to mlflow
    
except Exception as e:   # if any exception is there 
    raise e 

[2023-10-29 16:02:00,120: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-29 16:02:00,126: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-29 16:02:00,128: INFO: common: created directory at: artifacts]
Found 39 images belonging to 4 classes.
3/3 [==============================] - 11s 3s/step - loss: 51.0117 - accuracy: 0.2308
[2023-10-29 16:02:11,960: INFO: common: json file saved at: scores.json]


2023/10/29 16:02:30 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-29 16:02:32,697: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpkmr6uo8c\model\data\model\assets
[2023-10-29 16:02:33,779: INFO: builder_impl: Assets written to: C:\Users\user\AppData\Local\Temp\tmpkmr6uo8c\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/10/29 16:04:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
